In [1]:
# %pip install opencv-python deepface mysql-connector-python numpy scipy ulid-py

In [2]:
import cv2
from deepface import DeepFace
import mysql.connector
import os
import time
from datetime import datetime
import socket
import numpy as np
import json
from scipy.spatial.distance import cosine
import ulid  # For ULID face IDs

# ==== Configuration ====


In [3]:
CHECK_INTERVAL = 10
MAX_CAMERA_USE_TIME = 8
CAPTURED_FACES_DIR = "captured_faces"
EMBEDDING_MODEL = "ArcFace"
MATCH_THRESHOLD = 0.45  # Adjust as needed

# ==== Ensure storage directory exists ====


In [4]:
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# ==== Get Embeddings DB ====

In [5]:
def get_embeddings_db(cursor):
    cursor.execute("SELECT face_id, embedding FROM captured_snapshots WHERE embedding IS NOT NULL")
    known = []
    for face_id, emb_json in cursor.fetchall():
        try:
            emb = np.array(json.loads(emb_json), dtype=np.float64)
            known.append((face_id, emb))
        except Exception:
            continue
    return known

# ==== Get Face Embedding ====

In [6]:
def get_face_embedding(face_img):
    try:
        # DeepFace expects a file path or numpy array (BGR)
        reps = DeepFace.represent(face_img, model_name=EMBEDDING_MODEL, enforce_detection=False)
        if reps and isinstance(reps, list):
            emb = reps[0]['embedding']
            return np.array(emb, dtype=np.float64)
    except Exception as e:
        print(f"❌ Embedding error: {e}")
    return None

# ==== Match Face ID ====

In [7]:
def match_face_id(embedding, known, threshold=MATCH_THRESHOLD):
    for face_id, known_emb in known:
        dist = cosine(embedding, known_emb)
        if dist < threshold:
            return face_id
    return None

# ==== Generate ULID Face ID ====

In [8]:
# ==== Generate ULID Face ID ====
def generate_ulid():
    return str(ulid.new())

# ==== Assign and Reuse Face IDs ====

In [9]:
# def get_or_create_face_id(pc_name):
#     face_id_file = f"{pc_name}_face_id.txt"
#     if os.path.exists(face_id_file):
#         with open(face_id_file, "r") as f:
#             return f.read().strip()
#     else:
#         import uuid
#         face_id = f"{pc_name}_{uuid.uuid4().hex[:8]}"
#         with open(face_id_file, "w") as f:
#             f.write(face_id)
#         return face_id

# ====  Save to Database ====

In [15]:
def save_snapshot_to_db(self, face_id, pc_name, image_path, timestamp, embedding):
    sql = """
        INSERT INTO captured_snapshots (face_id, pc_name, image_path, timestamp, embedding)
        VALUES (%s, %s, %s, %s, %s)
    """
    emb_json = json.dumps(embedding.tolist()) if embedding is not None else None

    # Ensure timestamp is a datetime object
    if isinstance(timestamp, str):
        timestamp = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')

    self.cursor.execute(sql, (face_id, pc_name, image_path, timestamp, emb_json))
    self.db.commit()


# ==== Main Background Face Recognition Class ====


In [16]:
class BackgroundFaceCapture:
    def __init__(self):
        self.db = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="emotion_detection"
        )
        self.cursor = self.db.cursor()
        self.pc_name = socket.gethostname()
        ensure_dir(CAPTURED_FACES_DIR)

    def save_snapshot(self, face_img, face_id, timestamp, embedding):
        font = cv2.FONT_HERSHEY_SIMPLEX
        # Always use a Python datetime object for timestamp
        if not isinstance(timestamp, datetime):
            timestamp = datetime.now()

        timestamp_str = timestamp.strftime('%Y%m%d_%H%M%S')  # for filenames
        timestamp_text = f"Captured: {timestamp_str}"

        # Overlay timestamp on image
        cv2.putText(face_img, timestamp_text, (10, 30), font, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

        # Save image to file
        filename = f"{face_id}_{timestamp_str}.jpg"
        path = os.path.join(CAPTURED_FACES_DIR, filename)
        saved = cv2.imwrite(path, face_img)

        if saved:
            print(f"📸 Snapshot saved: {path}")
            # ✅ Pass datetime object directly (not string)
            save_snapshot_to_db(self, face_id, self.pc_name, path, timestamp, embedding)
        else:
            print("❌ Failed to save snapshot")


    def process_frame(self, frame):
        try:
            faces = DeepFace.extract_faces(frame, enforce_detection=False)
            if not faces:
                print("⚠️ No face detected.")
                return None

            # Load known embeddings from DB
            known = get_embeddings_db(self.cursor)

            for face in faces:
                area = face['facial_area']
                x, y, w, h = area['x'], area['y'], area['w'], area['h']
                pad_y = int(h * 0.6)
                pad_x = int(w * 0.4)
                x1 = max(0, x - pad_x)
                y1 = max(0, y - pad_y)
                x2 = min(frame.shape[1], x + w + pad_x)
                y2 = min(frame.shape[0], y + h + pad_y)
                face_img = frame[y1:y2, x1:x2]

                # Enhance clarity under glasses using CLAHE
                gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
                clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
                enhanced_gray = clahe.apply(gray)
                face_img_clahe = cv2.cvtColor(enhanced_gray, cv2.COLOR_GRAY2BGR)

                # Get embedding for this face
                embedding = get_face_embedding(face_img_clahe)
                if embedding is None:
                    print("⚠️ Could not extract embedding for face.")
                    continue

                # Try to match with known faces
                matched_id = match_face_id(embedding, known)
                if matched_id:
                    face_id = matched_id
                    print(f"✅ Matched existing face_id: {face_id}")
                else:
                    # New individual: use ULID
                    face_id = generate_ulid()
                    print(f"🆕 New face detected, assigned ULID: {face_id}")

                # Always use a proper datetime object for timestamp
                timestamp = datetime.now()
                self.save_snapshot(face_img_clahe, face_id, timestamp, embedding)

        except Exception as e:
            print(f"⚠️ Face processing error: {e}")

    def _try_get_camera(self):
        for i in range(3):
            cap = cv2.VideoCapture(i, cv2.CAP_DSHOW)
            if cap.isOpened():
                return cap
        print("❌ No available camera found.")
        return None

    def run(self):
        print("🔄 Background face capture started...")
        while True:
            cap = self._try_get_camera()
            if cap:
                start_time = time.time()
                while time.time() - start_time < MAX_CAMERA_USE_TIME:
                    ret, frame = cap.read()
                    if ret:
                        self.process_frame(frame)
                cap.release()
            time.sleep(CHECK_INTERVAL)

# ==== Start the service ====


In [17]:
if __name__ == "__main__":
    recognizer = None
    try:
        recognizer = BackgroundFaceCapture()
        recognizer.run()
    except KeyboardInterrupt:
        print("🛑 Background service stopped manually.")
    finally:
        if recognizer is not None:
            recognizer.db.close()

🔄 Background face capture started...
🆕 New face detected, assigned ULID: 01K363NP0KDX5DXENBYSFVBS79
📸 Snapshot saved: captured_faces\01K363NP0KDX5DXENBYSFVBS79_20250821_104425.jpg
🆕 New face detected, assigned ULID: 01K363NP0KDX5DXENBYSFVBS79
📸 Snapshot saved: captured_faces\01K363NP0KDX5DXENBYSFVBS79_20250821_104425.jpg
✅ Matched existing face_id: 01K363NP0KDX5DXENBYSFVBS79
📸 Snapshot saved: captured_faces\01K363NP0KDX5DXENBYSFVBS79_20250821_104426.jpg
✅ Matched existing face_id: 01K363NP0KDX5DXENBYSFVBS79
📸 Snapshot saved: captured_faces\01K363NP0KDX5DXENBYSFVBS79_20250821_104426.jpg
✅ Matched existing face_id: 01K363NP0KDX5DXENBYSFVBS79
📸 Snapshot saved: captured_faces\01K363NP0KDX5DXENBYSFVBS79_20250821_104428.jpg
✅ Matched existing face_id: 01K363NP0KDX5DXENBYSFVBS79
📸 Snapshot saved: captured_faces\01K363NP0KDX5DXENBYSFVBS79_20250821_104428.jpg
✅ Matched existing face_id: 01K362ZWTA8RSRNX0EFJM1C5SW
📸 Snapshot saved: captured_faces\01K362ZWTA8RSRNX0EFJM1C5SW_20250821_104429.jpg
✅ 